In [ ]:
import os
import torch
from torch import nn
from PIL import Image
from torchvision import transforms

In [ ]:
class Make_dataset:
    def __init__(self,path):
        self.path=path
        self.img=os.listdir(path)
        self.transforms=transforms.Compose([
            transforms.Resize((64,64)),
            transforms.ToTensor()])
    def __len__(self):
        return len(self.img)
#         return 1000
    def __getitem__(self,idx):
        X=Image.open(f'{self.path}/{self.img[idx]}')
#         X=transforms.Resize((64,64))(X)
#         X=transforms.ToTensor()(X)
        return self.transforms(X)

In [ ]:
dataset=Make_dataset("../input/celeba-dataset/img_align_celeba/img_align_celeba")

In [ ]:
dataset[100001]

In [ ]:
data_loader=torch.utils.data.DataLoader(dataset, batch_size=500)

In [ ]:
for x in data_loader:
    print (x.shape)
    break

In [ ]:
next(iter(data_loader))

In [ ]:
class Dicriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_0=nn.Sequential(
            nn.Conv2d(3,32,kernel_size=(3,3),padding=(1,1)),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.layer_1=torch.nn.Sequential(
            nn.Conv2d(32,64,kernel_size=(3,3),padding=(1,1)),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.layer_2=torch.nn.Sequential(
            nn.Conv2d(64,128,kernel_size=(3,3),padding=(1,1)),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.layer_3=torch.nn.Sequential(
            nn.Conv2d(128,256,kernel_size=(3,3),padding=(1,1)),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.layer_4=torch.nn.Sequential(
            nn.Conv2d(256,512,kernel_size=(3,3),padding=(1,1)),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.layer_5=torch.nn.Sequential(
            nn.Conv2d(512,1024,kernel_size=(3,3),padding=(1,1)),
            nn.BatchNorm2d(1024),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.layer_6=torch.nn.Sequential(
            nn.Linear(1024,512),
            nn.Linear(512, 2)
        )
    def forward(self, x):
        x = self.layer_0(x)
        x = self.layer_1(x)
        x = self.layer_2(x)
        x = self.layer_3(x)
        x = self.layer_4(x)
        x = self.layer_5(x)
#         print(x.shape)
        x = x.view(-1, 1024)
        x = self.layer_6(x)
        return x

In [ ]:
# disc=dicriminator()

In [ ]:
# disc(torch.ones((2,3,64,64)))

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_0=torch.nn.Sequential(
            nn.ConvTranspose2d(100, 64*8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(64*8),
            nn.ReLU()
        )
        self.layer_1=torch.nn.Sequential(
            nn.ConvTranspose2d(64*8, 64*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64*4),
            nn.ReLU()
        )
        self.layer_2=torch.nn.Sequential(
            nn.ConvTranspose2d(64*4, 64*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64*2),
            nn.ReLU()
        )
        self.layer_3=torch.nn.Sequential(
            nn.ConvTranspose2d(64*2, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        
        self.layer_4=torch.nn.Sequential(
            torch.nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            torch.nn.Tanh()
        )
    
    def forward(self, x):
        x = self.layer_0(x)
        x = self.layer_1(x)
        x = self.layer_2(x)
        x = self.layer_3(x)
        x = self.layer_4(x)
        return x

In [ ]:
# gen=Generator()

In [ ]:
# gen(torch.ones((2,100,1,1)))

In [ ]:
# criterion=nn.CrossEntropyLoss()

In [ ]:
# optimizer_D=torch.optim.Adam(disc.parameters(), lr = 0.0001)
# optimizer_G=torch.optim.Adam(gen.parameters(), lr = 0.0001)

In [ ]:
device = 'cuda'
gen = Generator().to(device)
disc = Dicriminator().to(device)
# state_dict = torch.load("model.pth", map_location = device)
# gen.load_state_dict(state_dict["generator"])
# disc.load_state_dict(state_dict["discriminator"])
criterion = nn.CrossEntropyLoss()
optimizer_G = torch.optim.Adam(gen.parameters(), lr = 0.0001)
optimizer_D = torch.optim.Adam(disc.parameters(), lr = 0.0001)

In [ ]:
gen(torch.ones((2, 100, 1, 1), device=device)).shape, disc(torch.ones((2, 3, 64, 64), device=device)).shape

In [ ]:
def Train(epoch):
    total_loss_G=0
    total_loss_D=0
    for x in data_loader:
        x=x.to(device)
        true_d=disc(x)
        y_1=torch.ones((len(x)),dtype=torch.long, device = device)
        true_loss_d=criterion(true_d, y_1)
        optimizer_D.zero_grad()
        true_loss_d.backward()
        
        
        noise=torch.randn((len(x),100,1,1),device = device)
        fake_image=gen(noise)
        false_d=disc(fake_image.detach())
        y_0=torch.zeros((len(x)),dtype=torch.long,device = device)
        false_loss_d=criterion(false_d, y_0)
        false_loss_d.backward()
        optimizer_D.step()
        
        output=disc(fake_image)
        loss_g=criterion(output, y_1)
        optimizer_D.zero_grad()
        loss_g.backward()
        optimizer_G.step()
        
#         print(f"Discriminator Loss: {true_loss_d + false_loss_d}, Generator Loss: {loss_g}")
        
        total_loss_D += true_loss_d.item() + false_loss_d.item()
        total_loss_G += loss_g.item()
        
#         print(f"Discriminator Loss: {true_loss_d + false_loss_d}, Generator Loss: {loss_g}")
        
    print(f"Epoch: {epoch}, Loss D: {total_loss_D/len(data_loader)}, Loss G: {total_loss_G/len(data_loader)}")
    return epoch, total_loss_D/len(data_loader), total_loss_G/len(data_loader)

In [ ]:
for epoch in range(1, 31): Train(epoch)

In [ ]:
noise = torch.randn((1, 100, 1, 1), device = device)
fake_image = gen(noise)
transforms.ToPILImage()(fake_image.squeeze()).resize((256, 256))

In [ ]:
torch.save({"generator": gen.state_dict(), "discriminator": disc.state_dict()}, "model.pth")

In [ ]:
# x=os.listdir('../input/celeba-dataset/img_align_celeba/img_align_celeba')

In [ ]:
x

In [ ]:
# from PIL import Image
# from torchvision import transforms

In [ ]:
# img=Image.open(f'../input/celeba-dataset/img_align_celeba/img_align_celeba/{x[0]}')

In [ ]:
img

In [ ]:
# np.array(img).shape

In [ ]:
# transforms.Resize((64,64))(img)

In [ ]:
# transforms.ToTensor()(img).shape

In [ ]:
# transforms.Compose([transforms.Resize((64,64)),transforms.ToTensor()])(img).shape

In [ ]:
def x():
#     for i in range (10):
        yield i

In [ ]:
# for j in x():
#     print (j)

In [ ]:
# y=x()

In [ ]:
# next(y)

In [ ]:
# def x():
#     yield 1
#     yield 2
#     yield 3
    

In [ ]:
# y=x()

In [ ]:
# next(y)